<a href="https://colab.research.google.com/github/gryffincoders/digit-recognition-app/blob/main/Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install transformers sentencepiece accelerate pypdf2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s eta 0:00:00


In [26]:
from transformers import pipeline
import PyPDF2
import re

In [27]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [28]:
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [29]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove excessive spaces
    text = re.sub(r'\n', ' ', text)  # Remove newlines
    return text.strip()

In [30]:
def analyze_financial_report(file_path):
    # Extract & clean text
    raw_text = extract_text_from_pdf(file_path)
    clean_data = clean_text(raw_text)

    # Hugging Face summarization
    summary = summarizer(clean_data, max_length=300, min_length=100, do_sample=False)[0]['summary_text']

    # Simple keyword-based extraction
    insights = {
        "Revenue": re.findall(r"Revenue[\w\s:]*\$?[\d,.]+", clean_data, re.IGNORECASE),
        "Profit": re.findall(r"Profit[\w\s:]*\$?[\d,.]+", clean_data, re.IGNORECASE),
        "Loss": re.findall(r"Loss[\w\s:]*\$?[\d,.]+", clean_data, re.IGNORECASE),
        "Expenses": re.findall(r"Expenses[\w\s:]*\$?[\d,.]+", clean_data, re.IGNORECASE)
    }

    return {
        "summary": summary,
        "insights": insights
    }

In [34]:
from google.colab import files
uploaded = files.upload()

file_path = list(uploaded.keys())[0]

Saving pdf downloadable - pdf downloadable.pdf to pdf downloadable - pdf downloadable.pdf


In [35]:
results = analyze_financial_report(file_path)


Your max_length is set to 300, but your input_length is only 134. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)


In [37]:
print("\n Financial Report Summary:\n")
print(results["summary"])

print("\n Key Insights:")
for key, value in results["insights"].items():
    print(f"{key}: {value if value else 'Not Found'}")


 Financial Report Summary:

Revenue $5,000,000 Cost of Goods Sold (COGS) $2,000k Gross Profit $3,000K Operating Expenses - Salaries and Wages $1,200k Marketing and Advertising $500k Research and Development $300k General & Administrative $200k. Operating Income $800,000 Other Income $50k Interest Expense $30k Profit Before Tax $820,000 Income Tax Expense (25%) $205,000 Net Profit (or Net Income) $615,000

 Key Insights:
Revenue: ['Revenue $5,000,000']
Profit: ['Profit $3,000,000', 'Profit Before Tax $820,000']
Loss: Not Found
Expenses: ['Expenses $2,200,000']
